
#Import Libraries

In [1]:
import pandas as pd

from google.colab import drive
# from sklearn.preprocessing import LabelEncoder
# from imblearn.under_sampling import RandomUnderSampler
# from collections import Counter
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.callbacks import EarlyStopping

#Cek Dataset(Optional)


### If u have 2 or more dataset and want to compare the label and the data of both

In [2]:
# drive.mount('/content/drive')

# # Path file di Google Drive
# file_path_1 = '/content/drive/MyDrive/capstone/diabetes_prediction_dataset1.csv'
# file_path_2 = '/content/drive/MyDrive/capstone/preprocessed_diabetes_data.csv'

# # Membaca file CSV dari Google Drive
# df1 = pd.read_csv(file_path_1)
# df2 = pd.read_csv(file_path_2)


# # Memeriksa apakah kedua dataset memiliki dimensi yang sama
# if df1.shape == df2.shape:
#     print("Dataset memiliki jumlah baris dan kolom yang sama.")

#     # Membandingkan apakah kedua dataset sama persis
#     if df1.equals(df2):
#         print("Isi kedua dataset sama persis.")
#     else:
#         print("Isi kedua dataset berbeda.")
#         # Menampilkan perbedaan secara lebih mendetail
#         comparison = df1.compare(df2)
#         print("Perbedaan antara kedua dataset:")
#         print(comparison)
# else:
#     print("Dataset memiliki jumlah baris atau kolom yang berbeda.")

#Data Gathering

In [3]:
drive.mount('/content/drive')

# Path file di Google Drive
file_path = '/content/drive/MyDrive/Dataset/dataset_berita_nlp_100.csv'

# Membaca file CSV dari Google Drive
df = pd.read_csv(file_path)

Mounted at /content/drive


#Preprocessing Data

In [4]:
df.head()

,id,judul,isi_berita,kategori
0,1,Tim Nasional Memenangkan Kejuaraan,Pertandingan final berjalan dramatis dengan go...,olahraga
1,2,Perusahaan Teknologi Meluncurkan Produk Baru,Keamanan siber ditingkatkan untuk menghindari ...,teknologi
2,3,Keamanan Siber Semakin Ditingkatkan,Sebuah perusahaan teknologi besar meluncurkan ...,teknologi
3,4,Tim Nasional Memenangkan Kejuaraan,Tim nasional berhasil mengalahkan lawan dalam ...,olahraga
4,5,Presiden Mengunjungi Daerah Terpencil,Partai politik utama sedang melakukan kampanye...,politik


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          100 non-null    int64 
 1   judul       100 non-null    object
 2   isi_berita  100 non-null    object
 3   kategori    100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [6]:
df.isna().sum()

,0
id,0
judul,0
isi_berita,0
kategori,0


In [7]:
print(df.isnull().sum())

id            0
judul         0
isi_berita    0
kategori      0
dtype: int64


In [9]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.9 MB/s eta 0:00:00


In [26]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Membuat stemmer menggunakan Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Mengambil daftar stopwords bahasa Indonesia
stop_words = set(stopwords.words("indonesian"))

def preprocess_text(text):
    # Mengubah teks menjadi huruf kecil
    text = text.lower()

    # Menghapus angka dan tanda baca
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus tanda baca
    text = re.sub(r'\d+', '', text)      # Menghapus angka

    # Tokenisasi kata
    tokens = word_tokenize(text)

    # Menghapus stopwords dan melakukan stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    # Menggabungkan kembali kata-kata yang sudah diproses menjadi sebuah teks
    return " ".join(tokens)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
df.head()

,id,judul,isi_berita,kategori
0,1,Tim Nasional Memenangkan Kejuaraan,Pertandingan final berjalan dramatis dengan go...,olahraga
1,2,Perusahaan Teknologi Meluncurkan Produk Baru,Keamanan siber ditingkatkan untuk menghindari ...,teknologi
2,3,Keamanan Siber Semakin Ditingkatkan,Sebuah perusahaan teknologi besar meluncurkan ...,teknologi
3,4,Tim Nasional Memenangkan Kejuaraan,Tim nasional berhasil mengalahkan lawan dalam ...,olahraga
4,5,Presiden Mengunjungi Daerah Terpencil,Partai politik utama sedang melakukan kampanye...,politik


In [28]:
df['clean_text'] = df['isi_berita'].apply(preprocess_text)
from sklearn.feature_extraction.text import TfidfVectorizer

# Membuat model TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df["clean_text"])  # df["clean_text"] adalah kolom teks yang sudah diproses


In [30]:
from sklearn.preprocessing import LabelEncoder

# Encoding kategori menjadi numerik
encoder = LabelEncoder()
df['kategori_encoded'] = encoder.fit_transform(df['kategori'])


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df["kategori_encoded"], test_size=0.2)

# Membuat model Naïve Bayes
model = MultinomialNB()

# Melatih model
model.fit(X_train, y_train)

# Prediksi pada data test
y_pred = model.predict(X_test)

# Evaluasi model
print("Akurasi Model: ", accuracy_score(y_test, y_pred))


Akurasi Model:  1.0


In [33]:
import joblib

# Simpan model yang telah dilatih
joblib.dump(model, "model.pkl")


['model.pkl']

In [34]:
# Simpan TF-IDF vectorizer
joblib.dump(tfidf, "tfidf.pkl")


['tfidf.pkl']